In [ ]:
!pip install neuralforecast

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.6/170.6 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.9/776.9 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 MB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.1/806.1 kB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.6/230.6 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 3.7 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import numpy as np
import torch
from typing import Optional, Union, Tuple


import torch.nn.functional as F
from torch.distributions import Distribution
from torch.distributions import Bernoulli, Normal, StudentT, Poisson, NegativeBinomial

from torch.distributions import constraints


from neuralforecast import NeuralForecast
from neuralforecast.models import TimesNet
from neuralforecast.tsdataset import TimeSeriesDataset
from neuralforecast.utils import AirPassengers, AirPassengersPanel, AirPassengersStatic
from neuralforecast.losses.numpy import rmse, mape
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay
from neuralforecast.losses.pytorch import MQLoss, DistributionLoss, MSE, MAE

In [ ]:
from numpy.random import seed
from random import randrange

In [ ]:
df = pd.read_excel("SP100_vol (clean).xlsx", index_col="Date",parse_dates=True)
columns=df.columns
df_static=pd.read_excel("Static_df.xlsx")
n_series=len(df.columns)
train = df.iloc[:int(len(df['NKE'])*0.7)]
validation_length=int(len(df['NKE'])*0.7)-int(len(df['NKE'])*0.5)
train=train.unstack().reset_index(name="Value").rename(columns={"level_0":"unique_id", "Date":"ds", "Value":"y"})

# Test

In [ ]:
test_length=int(len(df['NKE'])*0.3)
df1=df.unstack().reset_index(name="Value").rename(columns={"level_0":"unique_id", "Date":"ds", "Value":"y"})

In [ ]:
RMSE=[0.004866413301652508, 0.004981050841453135, 0.004903594101499435, 0.004779902429353342, 0.004864262530105924, 0.004911832422889443, 0.004805589793620455, 0.0047531121397578965, 0.0049530263448614035, 0.0048864083863249365, 0.004866430309210429, 0.004783100161599275, 0.004807269767484709, 0.004799238970400283, 0.004791607038157853, 0.004761754785169641]
MAE=[0.0029844076183406553, 0.003186729901430635, 0.0030596173773791624, 0.0029909346241860065, 0.003024778546377212, 0.0030561232078688844, 0.002989272872432878, 0.0029617751498278674, 0.0031497907973215564, 0.003064383593638764, 0.0030423756304134592, 0.0029441567605983016, 0.003045833336588304, 0.0030154618153803454, 0.0029705577168548424, 0.002969485717611593]

QLIKE=[0.044470867795815244, 0.04569334299223168, 0.04477649518775349, 0.04413943197671193, 0.04463354740620321, 0.04521924955664408, 0.04425548870758705, 0.043982674995798075, 0.04522475581371795, 0.04473493802505513, 0.0451759913886738, 0.044362314742322974, 0.044285141394337604, 0.04398314608221429, 0.04414582324224126, 0.04389408010249364]

MAPE=[0.22909140515418291, 0.25441263120152646, 0.23734299831837308, 0.23260532615730975, 0.23546107266910296, 0.2354986478500587, 0.23144200936431736, 0.22882317743411912, 0.2500036594977219, 0.2399882849504336, 0.23621565750588266, 0.22536255319187257, 0.2402464146594516, 0.23552167726248668, 0.22873428489749473, 0.22985133914564762]
H=5

for m in range (4):
  randomseed=randrange(129228148)
  model = TimesNet(h=H, input_size=15, hidden_size=90,
                   conv_hidden_size=90, top_k=5,
             num_kernels=6, encoder_layers=2, loss=MSE(),
                   max_steps=150,
             learning_rate=0.0001,
             batch_size=32,
             scaler_type='standard', random_seed=randomseed)
  fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
  forecasts = fcst.cross_validation(df=df1,val_size=H,static_df=df_static,n_windows=None, test_size=test_length-test_length%H,step_size=H)
  forecasts = forecasts.dropna()
  if "TimesNet-median" not in list(forecasts.columns.values):
    Y_hat=forecasts["TimesNet"].values
  else:
    Y_hat=forecasts["TimesNet-median"].values
  Y_true=forecasts["y"].values
  RMSE.append(np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true)))
  QLIKE.append((np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true)))
  MAE.append(np.sum(abs(Y_true-Y_hat))/len(Y_true))
  MAPE.append(np.sum(abs(Y_true-Y_hat)/(Y_true))/len(Y_true))

print(f"RMSE mean: {np.mean(RMSE)} | QLIKE mean: {np.mean(QLIKE)} | MAE mean: {np.mean(MAE)} | MAPE mean: {np.mean(MAPE)}")
print(f"RMSE std: {np.std(RMSE)} | QLIKE std: {np.std(QLIKE)} | MAE std: {np.std(MAE)} | MAPE std: {np.std(MAPE)}")
from scipy.stats import anderson
print(f"RMSE: {anderson(RMSE)}")
print(f"QLIKE: {anderson(QLIKE)}")
print(f"MAE: {anderson(MAE)}")
print(f"MAPE: {anderson(MAPE)}")

df_errors = pd.DataFrame({
    'RMSE': RMSE,
    'QLIKE': QLIKE,
    'MAE': MAE,
    'MAPE': MAPE
})

df_errors.to_excel("TimesNet forecasts.xlsx")

INFO:lightning_fabric.utilities.seed:Seed set to 68384319


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

...

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

RMSE mean: 0.00484890002576275 | QLIKE mean: 0.04460329800494087 | MAE mean: 0.003034952169981565 | MAPE mean: 0.2364178347168026
RMSE std: 6.400569546713066e-05 | QLIKE std: 0.0004900010772950165 | MAE std: 6.679755765344014e-05 | MAPE std: 0.00789791188700285
RMSE: AndersonResult(statistic=0.4378638906457262, critical_values=array([0.506, 0.577, 0.692, 0.807, 0.96 ]), significance_level=array([15. , 10. ,  5. ,  2.5,  1. ]), fit_result=  params: FitParams(loc=0.00484890002576275, scale=6.566845795943092e-05)
 success: True
 message: '`anderson` successfully fit the distribution to the data.')
QLIKE: AndersonResult(statistic=0.386431141432368, critical_values=array([0.506, 0.577, 0.692, 0.807, 0.96 ]), significance_level=array([15. , 10. ,  5. ,  2.5,  1. ]), fit_result=  params: FitParams(loc=0.04460329800494087, scale=0.0005027304978030913)
 success: True
 message: '`anderson` successfully fit the distribution to the data.')
MAE: AndersonResult(statistic=0.685805536047237, critical_